In [1]:
import numpy as np
import pandas as pd
import string
import re

In [3]:
df = pd.read_csv("./train_E6oV3lV.csv")

In [4]:
df.head(10)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
5,6,0,[2/2] huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...
7,8,0,the next school year is the year for exams.ð...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...
9,10,0,@user @user welcome here ! i'm it's so #gr8 !


In [5]:
df.shape

(31962, 3)

In [6]:
# NLTK
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [7]:
# Module-level global variables for the `tokenize` function below
PUNCTUATION = set(string.punctuation)
STOPWORDS = set(stopwords.words('english'))
STEMMER = PorterStemmer()

In [8]:
def cleaningText(text):
    # The following regex just strips of an URL (not just http), any punctuations, 
    # User Names or Any non alphanumeric characters.    
    preprocesstext1 = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(\d+)"," ",text).split()).strip().lower()
    
    #start replaceTwoOrMore
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    preprocesstext2 = pattern.sub(r"\1\1", preprocesstext1)    
    return preprocesstext2

In [9]:
def tokenize(text):    
    #print (text)
    tokens = word_tokenize(text) #decoding encoded strings
    #print ("Tokens: ", tokens)
    lowercased = [t.lower() for t in tokens]
    no_punctuation = []
    for word in lowercased:
        punct_removed = ''.join([letter for letter in word if not letter in PUNCTUATION])
        no_punctuation.append(punct_removed)
    no_stopwords = [w.strip() for w in no_punctuation if not w in STOPWORDS]
    stemmed = [STEMMER.stem(w) for w in no_stopwords]
    return [w for w in stemmed]

In [10]:
df["cText"] = df.apply(lambda x: cleaningText(x["tweet"]), axis = 1)

In [11]:
df.head()

,id,label,tweet,cText
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i can t use cause they ...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ur
4,5,0,factsguide: society now #motivation,factsguide society now motivation


In [12]:
df["token"] = df.apply(lambda x: tokenize(x["cText"]), axis = 1)

In [13]:
df.head()

,id,label,tweet,cText,token
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...,"[father, dysfunct, selfish, drag, kid, dysfunc..."
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i can t use cause they ...,"[thank, lyft, credit, use, caus, offer, wheelc..."
2,3,0,bihday your majesty,bihday your majesty,"[bihday, majesti]"
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ur,"[model, love, u, take, u, time, ur]"
4,5,0,factsguide: society now #motivation,factsguide society now motivation,"[factsguid, societi, motiv]"


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [15]:
n_features = 1000
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features)

In [17]:
tf = tf_vectorizer.fit_transform(df["cText"])

In [21]:
tf.shape

(31962, 1000)

In [22]:
type(tf)

scipy.sparse.csr.csr_matrix

In [23]:
from sklearn.naive_bayes import GaussianNB

In [24]:
gnb = GaussianNB()

In [27]:
y_pred = gnb.fit(tf.toarray(), df["label"]).predict(tf.toarray())

In [28]:
y_pred

array([1, 1, 0, ..., 0, 1, 1])

In [29]:
from sklearn.metrics import confusion_matrix

In [30]:
confusion_matrix(df["label"], y_pred)

array([[14527, 15193],
       [    3,  2239]])

In [ ]:
`